# 비지도 학습 기반 형태소 분석
* 본 파일은 한국어 임베딩 3.2을 공부하며 정리한 자료임을 밝힙니다.

## Reference
* lovit님의 soynlp tutorial을 따라하며 공부한 자료임을 밝혀둡니다.

## soynlp

* 데이터 패턴을 스스로 학습하는 비지도 학습 접근법을 지향
* soynlp 에서 제공하는 WordExtractor 나 NounExtractor 는 여러 개의 문서로부터 학습한 통계 정보를 이용하여 작동
* 비지도학습 기반 접근법들은 통계적 패턴을 이용하여 단어를 추출하기 때문에 하나의 문장 혹은 문서에서 보다는 어느 정도 규모가 있는 동일한 집단의 문서 (homogeneous documents) 에서 잘 작동
* 영화 댓글들이나 하루의 뉴스 기사처럼 같은 단어를 이용하는 집합의 문서만 모아서 Extractors 를 학습하면 좋음

### 문제점 인식

기존 명사 추출기의 단점은 새로운 단어, 즉 이전에 학습한 데이터에 등장하지 않은 단어는 잘 인식하지 못한다는 점이다.<br>
아래의 예시를 살펴보자.

In [2]:
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum, Mecab

kkma = Kkma()
okt = Okt()
hannanum = Hannanum()
mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.5.1


In [3]:
sent = "손흥민과 황희찬은 대한민국 축구의 미래입니다."
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('OKT   명사: ', okt.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))
print('Mecab 명사: ', mecab.nouns(sent))

꼬꼬마 명사:  ['손', '손흥민', '흥', '민', '황희찬', '대한', '대한민국', '민국', '축구', '미래']
OKT   명사:  ['손흥민', '황희', '찬', '대한민국', '축구', '미래']
한나눔 명사:  ['손흥민', '황희찬', '대한민국', '축구', '미래']
Mecab 명사:  ['손흥민', '황희', '찬', '대한민국', '축구', '미래']


손흥민과 황희찬은 명사임을 우리는 알고 있지만 학습 데이터에 많이 등장하지 않아 이를 명사로 제대로 인식하지 못함을 보여준다.

물론 Mecab의 경우에는 이를 직접 사용자 사전에 추가할 수 있지만 일일이 추가하는 것에는 한계가 있다.

또한 '보코하람' 같이 외국어가 들어오면 이를 분해하는 특징도 있다. '보코하람'은 단어로 알지 못하지만 '보', '코' 라는 것은 명사로 알고 있기 때문입니다.

In [9]:
sent = '보코하람 테러로 소말리아에서 전쟁이 있었어요'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('OKT   명사: ', okt.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))
print('Mecab 명사: ', mecab.nouns(sent))

꼬꼬마 명사:  ['보', '보코', '코', '테러', '소말리', '전쟁']
OKT   명사:  ['보코하람', '테러', '소말리아', '전쟁']
한나눔 명사:  ['보코하람', '테러', '소말리아', '전쟁']
Mecab 명사:  ['보코', '하람', '테러', '소말리아', '전쟁']


soynlp에서는 이러한 문제점을 보완하기 위해 L-R 구조를 이용해서 명사 추출을 하는 비지도 학습 방법을 제안한다. L-R 구조를 통해 L 옆에 등장하는 R의 분포는 L이 명사인지 아닌지를 판단하는 좋은 힌트를 얻을 수 있다.<br>
하지만 이는 일반화할 수 없는데, 왜냐하면 보은, 순은 등은 '은'으로 끝나지만 '은'을 제외한 '보', '순'이 그 자체로 의미를 가지는 명사라고 보기 힘들기 때문이다.<br>
어쨌든, 이 방법은 주어진 문서집합에서 어절들의 구조를 학습하여 그 주어진 문서집합의 명사를 추출한다. 학습데이터가 필요하지 않은 통계 기반의 unsupervised 학습방법이다.

## LRNounExtractor

In [4]:
from soynlp.noun import LRNounExtractor

noun_extractor = LRNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded


NounExtractor.train(sents)의 sents는 len(sents)를 할 수 있는 list 형식이다.<br> DoublespaceLineCorpus 는 한 문장이 하나의 문서이며, 한 문서 내의 문장 구분을 두 칸 띄어쓰기 형식으로 저장한 텍스트 형식이다. iter_sent=True 이면 문서가 아닌 문장 단위로 yield 를 수행한다.

튜토리얼에 사용된 데이터는 2016년 10월 20일의 뉴스로, 한글로 이루어진 223,357개의 문장이다 (soynlp의 tutorial 데이터와 동일)

In [5]:
from soynlp.utils import DoublespaceLineCorpus

corpus_fname = 'C:/Users/sbh0613/Desktop/NLP/ratsgo/my Preprocessing/2016-10-20.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
len(sentences)

223357

추출하고 싶은 명사의 noun score threshold와 명사의 최소 빈도수 (min count)를 parameter로 설정한다.<br>
LRNounExtractor는 점수를 반환하는데 이의 범위는 [-1,1]이다. 이 점수에 대해 noun score threshold를 적용하는 것이다.

In [18]:
%%time
nouns = noun_extractor.train_extract(
    sentences, # input은 DoublespaceLineCorpus가 끝난 애들.
    min_noun_score=0.3,
    min_noun_frequency=20
)

[Noun Extractor] scanning was done (L,R) has (52264, 26090) tokens
[Noun Extractor] building L-R graph was done000 / 223357 sents
[Noun Extractor] 14589 nouns are extracted
Wall time: 2min 30s


nouns 는 dict[str] = namedtuple 형식으로 return 된다. namedtuple 인 NounScore 에는 어절의 왼쪽에 등장한 횟수, 명사 점수, R set 이 알려진 feature 인 비율이 저장되어 있다.

In [23]:
nouns['뉴스']

NounScore_v1(frequency=8325, score=0.43977009340659345, known_r_ratio=0.052089295935890095)

In [24]:
words = ['박근혜', '우병우', '민정수석', '트와이스', '아이오아이']
for word in words:
    print('%s is noun? %r' % (word, word in nouns))

박근혜 is noun? True
우병우 is noun? True
민정수석 is noun? True
트와이스 is noun? False
아이오아이 is noun? True


기준을 바꿔가며 명사인지 판단할 수 있음.

In [27]:
noun_extractor.is_noun('트와이스', min_noun_score=0.2)

True

In [31]:
nouns['아이오아이']

NounScore_v1(frequency=270, score=0.9803828505747126, known_r_ratio=1.0)

## NewsNounExtractor

* 뉴스 데이터에서 좋은 성능을 낼 수 있도록 함.
* init에 입력하는 arguments는 동일.

In [32]:
from soynlp.noun import NewsNounExtractor

noun_extractor = NewsNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/sbh0613/anaconda/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment


In [33]:
nouns = noun_extractor.train_extract(sentences)

scan vocabulary ... 
done (Lset, Rset, Eojeol) = (658116, 363342, 403882)
predicting noun score was done                                        
before postprocessing 237871
_noun_scores_ 50196
checking hardrules ... done0 / 50196+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 36027
extracted 2365 compounds from eojeolss ... 87000 / 87714

In [34]:
words = ['박근혜', '우병우', '민정수석', 
         '트와이스', '아이오아이', '최순실',
         '최순실게이트', '게이트', '콘서트']

for word in words:
    if not word in nouns:
        continue
    score = nouns[word]
    print('%s: (score=%.3f, frequency=%d)' 
          % (word, score.score, score.frequency))

박근혜: (score=0.478, frequency=1507)
우병우: (score=0.757, frequency=721)
민정수석: (score=0.834, frequency=812)
아이오아이: (score=0.547, frequency=270)
최순실: (score=0.828, frequency=1878)
게이트: (score=0.745, frequency=307)
콘서트: (score=0.769, frequency=500)


## LRNounExtractor_v2

명사 추출을 하기 위해 lovit님이 여러 시도를 하였는데, 앞서 살펴본 extractor ver 1과 news noun extractor, 그리고 지금부터 살펴볼 extractor ver 2가 그것들이다.<br>
v1와 news noun extractor의 단점을 보완한 것이 ver 2이기 때문에 ver 2가 가장 좋은 성능을 낸다고 한다.

ver 2에서는 ver 1에 비해서 아래의 사항들이 개선되었다.<br>
version 2 에서는 (1) 명사 추출의 정확성을 높였으며, (2) 합성명사의 인식이 가능. 또한 (3) 명사의 빈도를 정확히 계산

In [11]:
import soynlp
print(soynlp.__version__)

from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

corpus_fname = 'C:/Users/sbh0613/Desktop/NLP/ratsgo/my Preprocessing/2016-10-20.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)

0.0.493


사용법은 ver 1와 비슷하다.<br>
train_extract 함수를 통하여 명사 점수를 계산할 수 있다.<br>
verbose mode 일 경우에는 학습 과정의 진행 상황이 출력된다.<br>
자세한 차이점은 lovit님의 블로그 lovit.github.io/nlp/2018/05/08/noun_extraction_ver2 을 참조하자.

아래와 같이 train, extract을 따로 할 수 있다.

In [13]:
%%time
# extract_compund는 합성 명사의 추출 여부!
noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)
noun_extractor.train(sentences)
nouns = noun_extractor.extract()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 403896 from 223357 sents. mem=0.154 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=4434442, mem=0.896 Gb
[Noun Extractor] batch prediction was completed for 119705 words
[Noun Extractor] checked compounds. discovered 70639 compounds
[Noun Extractor] postprocessing detaching_features : 109312 -> 92205
[Noun Extractor] postprocessing ignore_features : 92205 -> 91999
[Noun Extractor] postprocessing ignore_NJ : 91999 -> 90643
[Noun Extractor] 90643 nouns (70639 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.016 Gb                    
[Noun Extractor] 76.63 % eojeols are covered
Wall time: 1min 32s


아래와 같이 train, extract을 train_extract로 한번에 진행할 수 있다.<br>
이때, min_count와 minimum_noun_score는 train_extract에서, 또는 extract에서 조절할 수 있다.

In [16]:
%%time
nouns = noun_extractor.train_extract(sentences)
# nouns = noun_extractor.train_extract(sents, min_count=1, minimum_noun_score=0.3)

[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 403896 from 223357 sents. mem=1.072 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=4434442, mem=1.387 Gb
[Noun Extractor] batch prediction was completed for 119705 words
[Noun Extractor] checked compounds. discovered 70639 compounds
[Noun Extractor] postprocessing detaching_features : 109312 -> 92205
[Noun Extractor] postprocessing ignore_features : 92205 -> 91999
[Noun Extractor] postprocessing ignore_NJ : 91999 -> 90643
[Noun Extractor] 90643 nouns (70639 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.379 Gb                    
[Noun Extractor] 76.63 % eojeols are covered
Wall time: 1min 36s


nouns 는 {str: NounScore} 형식의 dict이다. 추출된 명사 단어에 대한 빈도수와 명사 점수가 namedtuple 인 NounScore 로 저장되어 있다.

ver 1과의 차이점으로, version 1 의 명사 추출기에서는 '뉴스'라는 left-side substring 의 빈도수를 명사의 빈도수로 이용하였습니다만, version 2 에서는 어절에서 '뉴스'가 실제로 명사로 이용된 경우만 카운팅 된다. '뉴스방송'과 같은 복합명사의 빈도수는 '뉴스'에 포함되지 않는다.

예를 들어, ver 1에서 '뉴스'의 frequency가 8325였지만 ver 2에서는 freq가 이보다 더 적어진다.

In [17]:
nouns['뉴스']

NounScore(frequency=4336, score=0.9548872180451128)

LRNounExtractor_v2._compounds_components 에는 복합 명사의 components 가 저장되어 있다. _compounds_components 는 {str:tuple of str} 형식이다.

In [19]:
print('복합 명사의 개수는 {0}개 입니다.'.format(len(noun_extractor._compounds_components)))

복합 명사의 개수는 70639개 입니다.


noun_extractor._compounds_components는 {복합 명사: 그 복합 명사를 이루는 명사들} 의 dictionary로 구성되어 있다.

In [24]:
list(noun_extractor._compounds_components.items())[:5]

[('잠수함발사탄도미사일', ('잠수함', '발사', '탄도미사일')),
 ('미사일대응능력위원회', ('미사일', '대응', '능력', '위원회')),
 ('글로벌녹색성장연구소', ('글로벌', '녹색성장', '연구소')),
 ('시카고옵션거래소', ('시카고', '옵션', '거래소')),
 ('대한민국특수임무유공', ('대한민국', '특수', '임무', '유공'))]

복합 명사도 nouns 에 포함되어 출력됩니다.

In [25]:
nouns['잠수함발사탄도미사일']

NounScore(frequency=1.0, score=18)

LRNounExtractor_v2.decompose_compound 는 입력된 str 가 복합 명사일 경우, 이를 단일 명사의 tuple 로 분해한다.

In [26]:
noun_extractor.decompose_compound('두바이월드센터시카고옵션거래소')

('두바이', '월드', '센터', '시카고', '옵션', '거래소')

복합명사가 아닌 경우에는 길이가 1 인 tuple 로 출력됩니다.

In [28]:
noun_extractor.decompose_compound('잠수함발사탄도미사일일까아닐까말까')

('잠수함발사탄도미사일일까아닐까말까',)

LRNounExtractor_v2 는 soynlp.utils 의 LRGraph 를 이용한다. 데이터의 L-R 구조를 살펴볼 수 있다.

In [30]:
noun_extractor.lrgraph.get_r('뉴스')

[('', 4186),
 ('1', 3511),
 ('1코리아', 2424),
 ('1스타', 352),
 ('센터', 106),
 ('제보', 99),
 ('투데이', 62),
 ('를', 54),
 ('테이', 50),
 ('랩', 40)]

topk=10 으로 설정되어 있다. topk < 0 으로 설정하면 모든 R set 이 출력된다..

In [31]:
noun_extractor.lrgraph.get_r('뉴스', topk=-1)

[('', 4186),
 ('1', 3511),
 ('1코리아', 2424),
 ('1스타', 352),
 ('센터', 106),
 ('제보', 99),
 ('투데이', 62),
 ('를', 54),
 ('테이', 50),
 ('랩', 40),
 ('룸', 37),
 ('팀', 35),
 ('쇼', 32),
 ('데스크', 29),
 ('로', 21),
 ('가', 18),
 ('는', 17),
 ('1과', 13),
 ('테이는', 9),
 ('랩을', 9),
 ('브리핑', 9),
 ('에', 8),
 ('속보팀', 8),
 ('파이터', 8),
 ('화면', 7),
 ('테이이자', 6),
 ('에서', 6),
 ('에디터', 6),
 ('1과의', 6),
 ('통신', 5),
 ('앵커', 5),
 ('테이와', 5),
 ('타파', 5),
 ('1스타에', 5),
 ('공장', 4),
 ('와', 4),
 ('와의', 4),
 ('1에', 4),
 ('콘텐츠팀', 4),
 ('의', 4),
 ('앤이슈', 4),
 ('앤이슈에서', 4),
 ('피드', 3),
 ('현장', 3),
 ('테이가', 3),
 ('들을', 3),
 ('8', 3),
 ('다', 3),
 ('제작과정에', 3),
 ('채널', 3),
 ('퀘어에서', 3),
 ('특급에서', 3),
 ('룸에서', 2),
 ('나', 2),
 ('도', 2),
 ('제작2부', 2),
 ('1번지', 2),
 ('분석', 2),
 ('테이리츠에', 2),
 ('래빗', 2),
 ('레터', 2),
 ('레터로', 2),
 ('그래픽', 2),
 ('에는', 2),
 ('룸은', 2),
 ('테이의', 2),
 ('브리핑에', 2),
 ('테이에', 2),
 ('앤이슈에', 2),
 ('8이', 2),
 ('에서만', 2),
 ('특급', 2),
 ('테스크', 1),
 ('타운', 1),
 ('타운을', 1),
 ('타파는', 1),
 ('코프', 1),
 ('멘트로', 1),
 ('멘트가

L-R 구조의 L parts 도 확인할 수 있다. 이 역시 topk=10 으로 기본값이 설정되어 있다.

In [33]:
noun_extractor.lrgraph.get_l('었다고')

[('있', 125),
 ('없', 76),
 ('만들', 37),
 ('늘', 32),
 ('맺', 29),
 ('열', 28),
 ('들', 19),
 ('입', 16),
 ('되', 14),
 ('줄', 14)]

## NounExtractor
* 통계를 기반으로 단어의 경계를 학습하는 비지도 학습
* Accessor Variety, Branching Entropy, Cohesion Score의 통계를 이용

In [36]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.word import WordExtractor

corpus_fname = 'C:/Users/sbh0613/Desktop/NLP/ratsgo/my Preprocessing/2016-10-20.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)

train은 substrings의 빈도수를 카운팅 하는 것이며, extract는 init에 들어가는 값을 기준으로 단어를 선택하여 준다.

In [38]:
%%time

from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.694 Gbse memory 0.768 Gb
all cohesion probabilities was computed. # words = 16942
all branching entropies was computed # words = 355061
all accessor variety was computed # words = 355061
Wall time: 2min 21s


In [39]:
print('총 {0}개의 단어 후보의 단어 점수가 계산되었습니다.'.format(len(words)))

총 9048개의 단어 후보의 단어 점수가 계산되었습니다.


words는 {word:Score} 형식의 dictionary이다. Score는 soynlp/word.py에 구현되어있는 namedtuple이다.

In [41]:
print('type: %s\n' % type(words['아이오아이']))
print(words['아이오아이'])

type: <class 'soynlp.word._word.Scores'>

Scores(cohesion_forward=0.30063636035733476, cohesion_backward=0, left_branching_entropy=3.0548011243339506, right_branching_entropy=2.766022241109869, left_accessor_variety=32, right_accessor_variety=22, leftside_frequency=270, rightside_frequency=0)


WordExtractor가 계산하는 것은 다양한 종류의 단어 가능 점수들입니다. 이를 잘 조합하여 원하는 점수를 만들 수도 있습니다. 즐겨쓰는 방법 중 하나는 cohesion_forward에 right_branching_entropy를 곱하는 것으로, (1) 주어진 글자가 유기적으로 연결되어 함께 자주 나타나고, (2) 그 단어의 우측에 다양한 조사, 어미, 혹은 다른 단어가 등장하여 단어의 우측의 branching entropy가 높다는 의미입니다. from lovit님

아래는 cohesion_forward와 right_branching_entropy을 곱하여 만든 word_score가 **높은** 순으로 단어와 freq, cohesion, entropy을 배열한 결과이다.

In [46]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
            )
         )

단어   (빈도수, cohesion, branching entropy)

으로     (1634, 0.953, 5.334)
까지     (654, 0.691, 5.349)
함께     (7946, 0.912, 5.053)
통해     (8471, 0.578, 5.278)
에서     (7494, 0.604, 5.187)
된다     (2681, 0.982, 4.675)
먼저     (1112, 0.903, 4.665)
면서     (1944, 0.458, 5.337)
밝혔다     (8360, 0.836, 4.651)
했다     (7070, 0.689, 4.795)
됐다     (2219, 0.750, 4.658)
또한     (2180, 0.440, 5.086)
같은     (4429, 0.568, 4.832)
됩니다     (247, 0.967, 4.272)
새로운     (2334, 0.578, 4.784)
말했다     (8345, 0.706, 4.540)
관계자는     (2942, 0.501, 4.860)
였다     (211, 0.632, 4.556)
때문에     (4742, 0.696, 4.436)
과정에서     (990, 0.497, 4.738)
겁니다     (518, 0.915, 4.106)
위해     (8888, 0.367, 5.016)
예정이다     (3586, 0.607, 4.476)
따라     (3669, 0.366, 4.977)
따르면     (3470, 0.589, 4.440)
합니다     (739, 0.421, 4.766)
왔다     (674, 0.604, 4.396)
냈다     (340, 0.659, 4.298)
설명했다     (2055, 0.612, 4.370)
너무     (1247, 0.711, 4.209)


Cohesion score, Branching Entropy, Accessor Variety 에 대하여 각각의 점수만 이용하고 싶은 경우에는 다음의 함수를 이용한다.

In [48]:
cohesion_scores = word_extractor.all_cohesion_scores()
cohesion_scores['아이오아이'] # (cohesion_forward, cohesion_backward)

all cohesion probabilities was computed. # words = 16942


(0.30063636035733476, 0)

In [49]:
branching_entropy = word_extractor.all_branching_entropy()
branching_entropy['아이오아이'] # (left_branching_entropy, right_branching_entropy)

all branching entropies was computed # words = 355061


(3.0548011243339506, 2.766022241109869)

In [50]:
accessor_variety = word_extractor.all_accessor_variety()
accessor_variety['아이오아이'] # (left_accessor_variety, right_accessor_variety)

all accessor variety was computed # words = 355061


(32, 22)